# Preprocessing pipeline

The goal of this pipeline is to produce data that is clean enough to be further analyzed in ERP or connectivity analyses. This pipeline is semi-automatic; manual steps are not necessarily required, but it is advised to revisit the cleaned epochs. ICA components have to be selected manually.

## Outline

Suggestions from this [paper](https://www.biorxiv.org/content/10.1101/240044v1.full.pdf)
- apply bandpass filter (0.5 - 45Hz) on continuous data
- create fixed length epochs 
- preliminary bad epoch rejection
- fit ICA on cleaned data segments
- mark components based on visual inspection
- apply ICA on epochs
- run [autoreject](https://www.sciencedirect.com/science/article/abs/pii/S1053811917305013) 
- find and interpolate bad sensors

## References

[1] A. Gramfort, M. Luessi, E. Larson, D. Engemann, D. Strohmeier, C. Brodbeck, R. Goj, M. Jas, T. Brooks, L. Parkkonen, M. Hämäläinen, MEG and EEG data analysis with MNE-Python, Frontiers in Neuroscience, Volume 7, 2013, ISSN 1662-453X

[2] Mainak Jas, Denis Engemann, Federico Raimondo, Yousra Bekhti, and Alexandre Gramfort, “Automated rejection and repair of bad trials in MEG/EEG.” In 6th International Workshop on Pattern Recognition in Neuroimaging (PRNI), 2016.

[3] Mainak Jas, Denis Engemann, Yousra Bekhti, Federico Raimondo, and Alexandre Gramfort. 2017. “Autoreject: Automated artifact rejection for MEG and EEG data”. NeuroImage, 159, 417-429.

[4] Bigdely-Shamlo, N., Mullen, T., Kothe, C., Su, K. M., & Robbins, K. A. (2015). The PREP pipeline: standardized preprocessing for large-scale EEG analysis. Frontiers in neuroinformatics, 9, 16.



## Install package requirements

Can be ignored if requirements are already installed.

In [ ]:
!pip install -r ../requirements.txt

## Import packages


```%matplotlib qt``` is the recommended backend for interactive visualization (can be slower); 
switch to ```%matplotlib inline``` for (faster) static plots

In [ ]:
import os
from pathlib import Path
from ipyfilechooser import FileChooser

from eeg_preprocessing.preprocessing import *
from eeg_preprocessing.utils.events import get_events_from_raw, create_epochs_from_events
from eeg_preprocessing.utils.io_raw import read_raw

from matplotlib import pyplot as plt
%matplotlib qt

In [ ]:
import sys
sys.path.append('..')

## Load raw data

In [ ]:
# Set base path to EEG data
base_path = '/Volumes/crnl-memo-hd/TMS_rewiring/Raw_data'

# Create folder for preprocessed and interim files
folder_name = 'preprocessed'
interim_path = os.path.join(base_path, folder_name)

# Use the widget to navigate to the experiment folder path and select an EEG file 
fc = FileChooser(base_path)
fc.filter_pattern = ['*.vhdr', '*.edf']

display(fc)

In [ ]:
# Load selected file
raw = read_raw(raw_file_path=fc.selected, add_info=False)
print(raw.info)

## Event processing

In [ ]:
# Extract triggers from raw instance
events = get_events_from_raw(raw)

resting_event_names = events.loc[events['event'].str.contains('rs_'), 'event'].tolist()
asrt_event_names = events.loc[events['event'].str.contains('asrt_'), 'event'].tolist()

In [ ]:
# Uncomment to show events
events.head(20)

## Cut raw data and create epochs based on triggers

### Create epochs

- bandpass filter the continuous data (0.5 - 45 Hz)
- create fixed length epochs (1 second)

In [ ]:
epochs = create_epochs_from_events(raw=raw, events=events)

## Run preprocessing


### 1.1. Preliminary epoch rejection

In [ ]:
epochs_faster = prepare_epochs_for_ica(epochs=epochs)

### 1.2. Run ICA

We run ICA for the resting and ASRT periods together; it will take a few minutes.
The parameters are: 32 ICA components using ["infomax"](https://mne.tools/stable/generated/mne.preprocessing.infomax.html) algorithm. 

When visualizing the components, it is recommended to subset the data (see below).

In [ ]:
ica = run_ica(epochs=epochs_faster)

In [ ]:
# Visualize components on epochs
# Subset epochs to reduce execution time
subset = resting_event_names + [asrt_event_names[0]]
# Exclude components by selecting them, right click on component name to visulize source:
ica.plot_sources(epochs_faster, start=0, stop=10)

In [ ]:
# After selecting the components to exclude, apply ICA to epochs
# Document the number of excluded components
ica.apply(epochs_faster)
epochs_faster.info['description'] = f'n_components: {len(ica.exclude)}'

### 1.3. Visualize ICA cleaned epochs (optional)

This step can be repeated after each preprocessing step, or you can also do a final inspection at the end. 

In [ ]:
epochs_faster.plot(n_epochs=10, scalings={'eeg': 20e-6}, title=raw.info['fid'])

In [ ]:
# Optional

# If you found a component that should have been excluded but it wasn't you can exclude it here:
ica.plot_sources(epochs_faster['rs_3_1'], start=0, stop=10)

In [ ]:
# Optional

# After selecting the components to exclude, apply ICA to epochs
# Document the number of excluded components
ica.apply(epochs_rs_faster)
epochs_rs_faster.info['description'] = f'n_components: {len(ica.exclude)}'

### 1.4. Save cleaned epochs (recommended)

In [ ]:
# Create path to epoch files
interim_epochs_path = os.path.join(interim_path, raw.info['condition'], 'epochs')
if not os.path.exists(interim_epochs_path):
    os.makedirs(interim_epochs_path)

# Save ICA cleaned epochs 
fid = epochs_faster.info['fid']
epochs_clean_fname = f'{fid}_ICA'
postfix = '-epo.fif.gz'
epochs_faster.save(os.path.join(interim_epochs_path, f'{epochs_clean_fname}{postfix}'), overwrite=True)

### 2.1. Run autoreject

In [ ]:
ar = run_autoreject(epochs_faster, n_jobs=11, subset=False)

In [ ]:
# Drop bad epochs (stage 1)

reject_log = ar.get_reject_log(epochs_faster)

epochs_autoreject = epochs_faster.copy().drop(reject_log.bad_epochs, reason='AUTOREJECT')

In [ ]:
# Drop bad epochs (stage 2) - after visual inspection
idx = np.where(np.count_nonzero(reject_log.labels, axis=1) > epochs_faster.info['nchan']/2)[0].tolist()

# # Plot just the bad epochs!
if idx: 
    epochs_faster[idx].plot(n_epochs=10,
                                scalings={'eeg': 20e-6},
                                n_channels=32)
    
epochs_autoreject.drop(idx, reason='AUTOREJECT')

In [ ]:
epochs_autoreject.drop(idx, reason='AUTOREJECT')

In [ ]:
# save clean epochs
fid = epochs_autoreject.info['fid']
epochs_clean_fname = f'{fid}_ICA_autoreject'
postfix = '-epo.fif.gz'
epochs_autoreject.save(os.path.join(interim_epochs_path, f'{epochs_clean_fname}{postfix}'), overwrite=True)

### 3. Run ransac

In [ ]:
epochs_ransac = run_ransac(epochs_autoreject)

In [ ]:
# inspect which sensors were interpolated (if any)
epochs_ransac.info

### 4. Final visual inspection

Mark epochs that should be dropped, select electrodes that should be interpolated etc.

In [ ]:
epochs_ransac

In [ ]:
epochs_ransac.plot(n_epochs=10,
                       n_channels=32,
                       # group_by='position',
                       scalings={'eeg': 20e-6})

In [ ]:
# if there are additional channels marked for interpolation, we can interpolate them here.

if epochs_ransac.info['bads']:
    bads_str = ', '.join(epochs_ransac.info['bads'])
    epochs_ransac.interpolate_bads()
    epochs_ransac.info.update(description=epochs_ransac.info['description'] + ', interpolated: ' + bads_str)

### 5. Set average reference

In [ ]:
epochs_ransac.set_eeg_reference()

### 6. Annotate continuous data


In [ ]:
start_times = [epochs.events[idx][0] / raw.info['sfreq'] 
               for idx, value in enumerate(epochs_ransac.drop_log) if value]

duration = (epochs_ransac.events[1][0] - epochs_ransac.events[0][0]) / raw.info['sfreq'] 

raw.annotations.append(onset=start_times,
                       duration=[duration] * len(start_times),
                       description='BAD_auto')

In [ ]:
# Create path to annotated files
annotated_raw_path = os.path.join(interim_path, raw.info['condition'], 'raw')
if not os.path.exists(annotated_raw_path):
    os.makedirs(annotated_raw_path)

# Save annotated continuous data
fid = raw.info["fid"]
raw_annotated_fname = f'{fid}_bad_annotated'
postfix = '-raw.fif.gz'
raw.save(os.path.join(annotated_raw_path, f'{raw_annotated_fname}{postfix}'), overwrite=True)

### 7. Save cleaned epochs

#### 7.1. Resting period before ASRT

In [ ]:
# Create path to annotated files
epochs_rs_path = os.path.join(interim_path, raw.info['condition'], 'epochs_rs')
if not os.path.exists(epochs_rs_path):
    os.makedirs(epochs_rs_path)

rs_period_name = f'rs_{raw.info["num_day"]}_1'
fid = f'{raw.info["subject"]}_{raw.info["condition"]}_{rs_period_name}'
epochs_clean_fname = f'{fid}_ICA_autoreject_ransac'
postfix = '-epo.fif.gz'

epochs_ransac[rs_period_name].save(os.path.join(epochs_rs_path, f'{epochs_clean_fname}{postfix}'), overwrite=True)

#### 7.2. Resting period before ASRT

In [ ]:
rs_period_name = f'rs_{raw.info["num_day"]}_2'
fid = f'{raw.info["subject"]}_{raw.info["condition"]}_{rs_period_name}'
epochs_clean_fname = f'{fid}_ICA_autoreject_ransac'
postfix = '-epo.fif.gz'

epochs_ransac[rs_period_name].save(os.path.join(epochs_rs_path,
                                                f'{epochs_clean_fname}{postfix}'), overwrite=True)

#### 7.3. ASRT

In [ ]:
# Create path to annotated files
epochs_asrt_path = os.path.join(interim_path, raw.info['condition'], 'epochs_asrt')
if not os.path.exists(epochs_asrt_path):
    os.makedirs(epochs_asrt_path)

In [ ]:
for sequence, periods in events.groupby('sequence')['event'].apply(set).to_dict().items():
    #epochs_to_merge = [epochs_ransac[period] for period in periods]
    #merged_epochs = mne.concatenate_epochs(epochs_to_merge, offset=True)
    fid = f'{raw.info["subject"]}_{raw.info["condition"]}_asrt_{raw.info["num_day"]}_{sequence}'
    epochs_clean_fname = f'{fid}_ICA_autoreject_ransac'
    postfix = '-epo.fif.gz'
    
    epochs_ransac[sorted(set(periods))].save(os.path.join(epochs_asrt_path,
                                                          f'{epochs_clean_fname}{postfix}'), overwrite=True)

In [ ]:
epochs_ransac

In [ ]:
# cleanup from memory
del raw, epochs, epochs_autoreject, epochs_ransac

plt.close('all')